Day 3

In [2]:
pip install scikit-learn pandas numpy

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

In [ ]:
# Loading the dataset (products.csv)
df = pd.read_csv('C:\\Users\\acer\\Desktop\\Infinity AI Work\\InfinityStyleVerse\\data\\raw\\products.csv')

In [15]:
df.head()

,title,category,color,material,rating,description
0,Slim Fit Blazer,Formal,Navy,Cotton,4.6,Stylish modern blazer for professionals.
1,Floral Midi Dress,Dress,Red,Polyester,4.3,Elegant floral dress perfect for summer outings.
2,Denim Jacket,Casual,Blue,Denim,4.7,Classic denim jacket with a relaxed fit.
3,Silk Scarf,Accessories,Green,Silk,4.2,Luxurious silk scarf to elevate any outfit.
4,Leather Ankle Boots,Footwear,Black,Leather,4.8,Sleek leather boots for a chic look.


In [16]:
descriptions = df['description'].values

In [17]:
# Creating the TfidfVectorizer 
tfidf = TfidfVectorizer()

# Converting the descriptions into numerical features
matrix = tfidf.fit_transform(descriptions)

In [18]:
# Calculating the cosine similarity matrix
similarities = cosine_similarity(matrix)

Day 4

In [20]:
# Function to get the top 3 most similar products for a given index
def get_similar_products(index):
    scores = list(enumerate(similarities[index]))
    scores = sorted(scores, key=lambda x: x[1], reverse=True)
    top_3_products = scores[1:4]

    # Returns the title and the score for the top 3 products
    return [(df['title'][i], score) for i, score in top_3_products]

Day 5

In [22]:
# Testing by giving a product index

product_index = 0

print(f"Products similar to {df['title'][product_index]}:")
similar_products = get_similar_products(product_index)
for title, score in similar_products:
    print(f"- {title} (Similarity: {score:.2f})")

Products similar to Slim Fit Blazer:
- Linen Blazer (Similarity: 0.26)
- Suede Boots (Similarity: 0.18)
- Suede Sneakers (Similarity: 0.18)


In [23]:
product_index = 4

print(f"Products similar to {df['title'][product_index]}:")
similar_products = get_similar_products(product_index)
for title, score in similar_products:
    print(f"- {title} (Similarity: {score:.2f})")

Products similar to Leather Ankle Boots:
- Suede Boots (Similarity: 0.62)
- Leather Jacket (Similarity: 0.27)
- Leather Gloves (Similarity: 0.18)


In [24]:
product_index = 10

print(f"Products similar to {df['title'][product_index]}:")
similar_products = get_similar_products(product_index)
for title, score in similar_products:
    print(f"- {title} (Similarity: {score:.2f})")

Products similar to Knit Sweater:
- Knit Beanie (Similarity: 0.47)
- Knit Vest (Similarity: 0.38)
- Knit Socks (Similarity: 0.35)


In [27]:
# Testing by giving a product title

In [28]:
# Function to get the index

def get_product_index(title):
    try:
        return df.index[df['title'] == title].tolist()[0]
    except IndexError:
        return None  # Returns None if no title was found

In [33]:
# User inputs
user_title = input("Enter a product title: ")

# Finding the index of the entered product
index = get_product_index(user_title)

if index is not None:
    print(f"Looking for recommendations for: {user_title}")
    similar_products = get_similar_products(index)
    for title, score in similar_products:
        print(f"- {title} (Similarity: {score:.2f})")
else:
    print("Product was not found. Please check the product title and try again.")

Looking for recommendations for: Knit Sweater
- Knit Beanie (Similarity: 0.47)
- Knit Vest (Similarity: 0.38)
- Knit Socks (Similarity: 0.35)
